In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import warnings
warnings.filterwarnings('ignore')

/kaggle/input/pneumoniamnist/aucm_trained_model_pneumoniamnist_gaussian.pth
/kaggle/input/model-breastmnist/aucm_trained_model_breastmnist_gaussian.pth


In [2]:
%%capture
!pip install libauc==1.2.0
!pip install medmnist
!pip install torchio

In [3]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

import medmnist
from medmnist import INFO, Evaluator

from libauc.models import resnet18
from libauc.sampler import DualSampler
from libauc.metrics import auc_prc_score

import torchvision.transforms as transforms
from torch.utils.data import Dataset
from PIL import Image, ImageFilter
from torchio import Image

In [4]:
print(f"MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

MedMNIST v2.2.1 @ https://github.com/MedMNIST/MedMNIST/


In [5]:
#data_flag = 'breastmnist'
data_flag = 'pneumoniamnist'
#data_flag = 'chestmnist'

download = True

#NUM_EPOCHS = 3
BATCH_SIZE = 128
#lr = 0.001

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

In [6]:
print(info)
print(n_channels)
print(n_classes)
print(DataClass)

{'python_class': 'PneumoniaMNIST', 'description': 'The PneumoniaMNIST is based on a prior dataset of 5,856 pediatric chest X-Ray images. The task is binary-class classification of pneumonia against normal. We split the source training set with a ratio of 9:1 into training and validation set and use its source validation set as the test set. The source images are gray-scale, and their sizes are (384−2,916)×(127−2,713). We center-crop the images and resize them into 1×28×28.', 'url': 'https://zenodo.org/record/6496656/files/pneumoniamnist.npz?download=1', 'MD5': '28209eda62fecd6e6a2d98b1501bb15f', 'task': 'binary-class', 'label': {'0': 'normal', '1': 'pneumonia'}, 'n_channels': 1, 'n_samples': {'train': 4708, 'val': 524, 'test': 624}, 'license': 'CC BY 4.0'}
1
2
<class 'medmnist.dataset.PneumoniaMNIST'>


In [7]:
from torchio.transforms import RandomAffine, RandomFlip, RandomNoise, RandomGamma
class ImageDataset(Dataset):
    def __init__(self, images, targets, image_size=28, crop_size=24, mode='train', kernel_size=3):
        self.images = images.astype(np.uint8)
        self.targets = targets
        self.mode = mode

        
        self.transform_train = transforms.Compose([
                                                    transforms.ToTensor(),
                                                    transforms.Resize((image_size, image_size)),
                                                    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.2), shear=0.2),
                                                    transforms.RandomApply([transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)], p=0.5),
                                                    transforms.RandomHorizontalFlip(p=0.5),
                                                    transforms.RandomVerticalFlip(p=0.5),
                                                    transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.2),
                                                ])

        self.transform_test = transforms.Compose([
                             transforms.ToTensor(),
                             #transforms.GaussianBlur(kernel_size=(kernel_size, kernel_size), sigma=(0.1, 2.0)),
                             #transforms.BilateralFilter(diameter=5, sigma_color=0.1, sigma_space=15),
                             transforms.Resize((image_size, image_size)),
                              ])
        
        
        # for loss function
        self.pos_indices = np.flatnonzero(targets==1)
        self.pos_index_map = {}
        for i, idx in enumerate(self.pos_indices):
            self.pos_index_map[idx] = i

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        target = self.targets[idx]
        image = Image.fromarray(image.astype('uint8'))
        #image = torch.from_numpy(image).unsqueeze(0) 
        if self.mode == 'train':
           idx = self.pos_index_map[idx] if idx in self.pos_indices else -1
           #image = image.filter(ImageFilter.GaussianBlur(radius=3))
           #image = image.filter(ImageFilter.BLUR)
           #image = image.filter(ImageFilter.UnsharpMask(radius=3, percent=150, threshold=3))

           image = self.transform_train(image)
        else:
           #image = image.filter(ImageFilter.GaussianBlur(radius=3))
           #image = image.filter(ImageFilter.BLUR)
           #image = image.filter(ImageFilter.UnsharpMask(radius=3, percent=150, threshold=3))
           image = self.transform_test(image)
        return idx, image, target 

In [8]:
train_dataset = DataClass(split='train', download=download)
print(train_dataset[0])
test_dataset = DataClass(split='test', download=download)
print(test_dataset[0])

100%|██████████| 4170669/4170669 [00:07<00:00, 534615.41it/s]

(<PIL.Image.Image image mode=L size=28x28 at 0x7A2B636D9A20>, array([1]))
Using downloaded and verified file: /root/.medmnist/pneumoniamnist.npz
(<PIL.Image.Image image mode=L size=28x28 at 0x7A2B636D9A20>, array([1]))


In [9]:
train_images = np.array([np.asarray(image) for (image, target) in train_dataset])
train_labels = np.array([target for (image, target) in train_dataset])
print(type(train_images[0]))

<class 'numpy.ndarray'>


In [10]:
test_images = np.array([np.asarray(image) for (image, target) in test_dataset])
test_labels = [target for (image, target) in test_dataset]
print(type(test_images[0]))

<class 'numpy.ndarray'>


In [11]:
test_images[0].shape

(28, 28)

In [12]:
batch_size = 32
sampling_rate = 0.5

trainSet = ImageDataset(train_images, train_labels)
trainSet_eval = ImageDataset(train_images, train_labels, mode='test')
testSet = ImageDataset(test_images, test_labels, mode='test')

sampler = DualSampler(trainSet, batch_size, sampling_rate=sampling_rate)
train_loader = torch.utils.data.DataLoader(trainSet, batch_size=batch_size, sampler=sampler, num_workers=2)
train_loader_at_eval = torch.utils.data.DataLoader(trainSet_eval, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = torch.utils.data.DataLoader(testSet, batch_size=batch_size, shuffle=False, num_workers=2)

In [13]:
print(train_dataset)
print("===================")
print(test_dataset)

Dataset PneumoniaMNIST (pneumoniamnist)
    Number of datapoints: 4708
    Root location: /root/.medmnist
    Split: train
    Task: binary-class
    Number of channels: 1
    Meaning of labels: {'0': 'normal', '1': 'pneumonia'}
    Number of samples: {'train': 4708, 'val': 524, 'test': 624}
    Description: The PneumoniaMNIST is based on a prior dataset of 5,856 pediatric chest X-Ray images. The task is binary-class classification of pneumonia against normal. We split the source training set with a ratio of 9:1 into training and validation set and use its source validation set as the test set. The source images are gray-scale, and their sizes are (384−2,916)×(127−2,713). We center-crop the images and resize them into 1×28×28.
    License: CC BY 4.0
Dataset PneumoniaMNIST (pneumoniamnist)
    Number of datapoints: 624
    Root location: /root/.medmnist
    Split: test
    Task: binary-class
    Number of channels: 1
    Meaning of labels: {'0': 'normal', '1': 'pneumonia'}
    Number of

In [ ]:
#print(train_dataset[0][0].shape)

# # montage
train_dataset.montage(length=20)

**Checking Imbalance in Dataset**

In [16]:
lables = train_dataset.labels
cnt1 = 0
cnt0 = 0
for i in range(len(lables)):
  if lables[i][0] == 0:
    cnt0+=1
  else:
    cnt1+=1
print("num of label 0 data",cnt0)
print("num of label 1 dat",cnt1)
print("minority data percent",cnt0*100/(cnt0+cnt1))


num of label 0 data 1214
num of label 1 dat 3494
minority data percent 25.78589634664401


In [17]:
from libauc.losses import AUCMLoss, CrossEntropyLoss
from libauc.optimizers import PESG, Adam
# from libauc.models import densenet121 as DenseNet121


from PIL import Image
from torch.utils.data import Dataset
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
import torchvision.models as models
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [18]:
#configure GPU
device = torch.device(0 if torch.cuda.is_available() else 'cpu')

In [19]:
import warnings
warnings.filterwarnings("ignore")

In [21]:
import os, random
def set_all_seeds(SEED):
    # REPRODUCIBILITY
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


Pretraining

In [25]:
# paramaters
SEED = 123
BATCH_SIZE = 32
lr = 1e-4
weight_decay = 1e-5
num_classes=1

# model
model = models.resnet18(pretrained=True)
# model = models.resnet18(pretrained=False)

input_channel = model.fc.in_features
#for param in model.parameters():
    #param.requires_grad = False
#model.fc = nn.Sequential(nn.Linear(input_channel, 128),
 #                        nn.ReLU(),
  #                       nn.Linear(128, num_classes),
   #                      nn.Sigmoid()
    #                     )
model.fc = nn.Sequential(nn.Linear(input_channel, 128),
                                 nn.ReLU(),
                                 nn.Dropout(p=0.2),
                                 nn.Linear(128, 32),
                                 nn.ReLU(),
                                 nn.Dropout(p=0.1),
                                 nn.Linear(32, num_classes),
                                 nn.Sigmoid()
                                 )


model = model.to(device)

# create a binary cross-entropy loss function
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = ReduceLROnPlateau(optimizer, patience=3,  verbose=True, factor=0.5, 
                              threshold=0.001, min_lr=0.00001, mode = 'max')

# training
best_val_auc = 0 
for epoch in range(20):
    for idx, (index, data, labels) in enumerate(train_loader):
      train_data, train_labels = data.to(device), labels.to(device)
      train_data=train_data.repeat(1,3,1,1)
#       train_labels=train_labels.float()

#       train_data, train_labels  = train_data.to(device), train_labels.to(device)
      # break
      #train_data = torch.tensor(train_data,requires_grad=False)
      y_pred = model(train_data)
      
      #y_pred=torch.where(y_pred > 0.5, torch.tensor([1.], device=device), torch.tensor([0.], device=device))
      #y_pred=torch.where(y_pred > 0.5, torch.tensor([1.], device=device), torch.tensor([0.], device=device))

      #print(y_pred)
      #y_pred = [1 if x > 0.5 else 0 for x in y_pred]
      loss = criterion(y_pred, train_labels.float())
      #print("Training Loss= ", loss.item())
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
    print(f'Validating epoch: {epoch+1} Loss: {loss.item()}')  
      # validation  
      #if idx % 100 == 0:
      #if (1):
    model.eval()
    val_auc_mean = 0
    with torch.no_grad():    
         val_pred = []
         val_true = [] 
         for jdx, (index, data, targets) in enumerate(train_loader_at_eval):
             val_data, val_labels = data.to(device), targets.to(device)
             val_data=val_data.repeat(1,3,1,1)
#              val_labels=val_labels.float()
#              val_data = val_data.to(device)
             y_pred = model(val_data)
             val_pred.append(y_pred.cpu().detach().numpy())
             val_true.append(val_labels.cpu().detach().numpy())
     
         val_true = np.concatenate(val_true)
         val_pred = np.concatenate(val_pred)
         #val_pred = [1 if x > 0.5 else 0 for x in val_pred]
         val_auc_mean =  roc_auc_score(val_true, val_pred) 
         #model.train
         if best_val_auc < val_auc_mean:
            best_val_auc = val_auc_mean
            torch.save(model.state_dict(), 'ce_pmnist_pretrained_model.pth')
         print ('Epoch=%s, BatchID=%s, Val_AUC=%.4f, Best_Val_AUC=%.4f'%(epoch, idx, val_auc_mean, best_val_auc) )
    scheduler.step(val_auc_mean)

Validating epoch: 1 Loss: 0.19362440705299377
Epoch=0, BatchID=217, Val_AUC=0.9765, Best_Val_AUC=0.9765
Validating epoch: 2 Loss: 0.13681606948375702
Epoch=1, BatchID=217, Val_AUC=0.9884, Best_Val_AUC=0.9884
Validating epoch: 3 Loss: 0.1836807280778885
Epoch=2, BatchID=217, Val_AUC=0.9905, Best_Val_AUC=0.9905
Validating epoch: 4 Loss: 0.22752372920513153
Epoch=3, BatchID=217, Val_AUC=0.9920, Best_Val_AUC=0.9920
Validating epoch: 5 Loss: 0.1392650455236435
Epoch=4, BatchID=217, Val_AUC=0.9917, Best_Val_AUC=0.9920
Validating epoch: 6 Loss: 0.1492317169904709
Epoch=5, BatchID=217, Val_AUC=0.9932, Best_Val_AUC=0.9932
Validating epoch: 7 Loss: 0.041403062641620636
Epoch=6, BatchID=217, Val_AUC=0.9928, Best_Val_AUC=0.9932
Validating epoch: 8 Loss: 0.29268425703048706
Epoch=7, BatchID=217, Val_AUC=0.9936, Best_Val_AUC=0.9936
Validating epoch: 9 Loss: 0.10858766734600067
Epoch=8, BatchID=217, Val_AUC=0.9928, Best_Val_AUC=0.9936
Validating epoch: 10 Loss: 0.40643495321273804
Epoch=9, BatchID=21

In [26]:
    print(f'Testing ...')  
    PATH = 'ce_pmnist_pretrained_model.pth' 
    model_state_dict = torch.load(PATH)
    #model = MyModel()  # Create an instance of your model
    model.load_state_dict(model_state_dict)  # Load the saved parameters into the model
    with torch.no_grad():    
         test_pred = []
         test_true = [] 
         for jdx, (index, data, targets) in enumerate(test_loader):
             test_data, test_labels = data.to(device), targets.to(device)
             test_data=test_data.repeat(1,3,1,1)
#              test_labels=test_labels.float()
#              test_data = test_data.to(device)
             y_pred = model(test_data)
             test_pred.append(y_pred.cpu().detach().numpy())
             test_true.append(test_labels.cpu().detach().numpy())
     
         test_true = np.concatenate(test_true)
         test_pred = np.concatenate(test_pred)
         #test_pred = [1 if x > 0.5 else 0 for x in test_pred]
         test_auc_mean =  roc_auc_score(test_true, test_pred) 
        # model.train
        # if best_val_auc < val_auc_mean:
        #    best_val_auc = val_auc_mean
        #    #torch.save(model.state_dict(), 'ce_pretrained_model.pth')
         print('Test_AUC=%.4f'%( test_auc_mean))

Testing ...
Test_AUC=0.9728


AUCM Optimization

In [29]:
from ignite.engine import *
from ignite.handlers import *
from ignite.metrics import *
from ignite.utils import *
from ignite.contrib.metrics.regression import *
from ignite.contrib.metrics import *
from torch.optim.lr_scheduler import ExponentialLR


# parameters
class_id = 1

# paramaters
SEED = 123
BATCH_SIZE = 32 #[16, 32, 64, 128]
#imratio = train_dataset.imratio
lr =0.01#0.05# 0.04#0.05 # using smaller learning rate is better [0.001, 0.01, 0.05, 0.1]
epoch_decay = 2e-3#1e-4
weight_decay = 1e-5#1e-5
margin = 1.0

# model
set_all_seeds(SEED)
# model
model = models.resnet18(pretrained=False)
input_channel = model.fc.in_features
#for param in model.parameters():
    #param.requires_grad = False
# model.fc = nn.Sequential(nn.Linear(input_channel, 128),
#                          nn.ReLU(),
#                          nn.Linear(128, num_classes),
#                          nn.Sigmoid()
#                          )

model.fc = nn.Sequential(nn.Linear(input_channel, 128),
                                 nn.ReLU(),
                                 nn.Dropout(p=0.2),
                                 nn.Linear(128, 32),
                                 nn.ReLU(),
                                 nn.Dropout(p=0.1),
                                 nn.Linear(32, num_classes),
                                 nn.Sigmoid()
                                 )


model = model.to(device)

#model = models.resnet18(pretrained=False)
#
#model.fc = nn.Sequential(
#    nn.Linear(512, 1),
#    nn.Sigmoid()
#)
#model = model.cuda()


# load pretrained model
if True:
  PATH = 'ce_pmnist_pretrained_model.pth' 
  state_dict = torch.load(PATH)
  state_dict.pop('classifier.weight', None)
  state_dict.pop('classifier.bias', None) 
  model.load_state_dict(state_dict, strict=False)


# define loss & optimizer
loss_fn = AUCMLoss()
optimizer = PESG(model, 
                 loss_fn=loss_fn, 
                 lr=lr, 
                 margin=margin, 
                 epoch_decay=epoch_decay, 
                 weight_decay=weight_decay)
#optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
#lr_scheduler_opt = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)
#optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = ReduceLROnPlateau(optimizer, patience=2,  verbose=True, factor=0.5, 
                              threshold=0.001, min_lr=0.00001, mode = 'max')
#torch_lr_scheduler = ExponentialLR(optimizer=optimizer, gamma=0.98)
#scheduler = create_lr_scheduler_with_warmup(torch_lr_scheduler,
 #                                           warmup_start_value=0.05,
  #                                          warmup_end_value=0.1,
   #                                         warmup_duration=2)

best_val_auc = 0
for epoch in range(30):
  if epoch%10 == 0 and epoch > 0:
      optimizer.update_regularizer(decay_factor=10)
  #lr_scheduler_opt.step()
  
  for idx, (index, data, labels) in enumerate(train_loader):
      train_data, train_labels = data.to(device), labels.to(device)
      train_data=train_data.repeat(1,3,1,1)
#       train_labels=train_labels.float()

#       train_data, train_labels = train_data.to(device), train_labels.to(device)
      y_pred = model(train_data)
      #y_pred = torch.sigmoid(y_pred)
      #y_pred=torch.where(y_pred > 0.5, torch.tensor([1.], device=device), torch.tensor([0.], device=device))
      #y_pred = [1 if x > 0.5 else 0 for x in y_pred]
      #print(y_pred)
      loss = loss_fn(y_pred, train_labels.float())
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # validation
  #if idx % 400 == 0:
  print(f'Validating epoch: {epoch+1} Loss: {loss.item()}')
  model.eval()
  val_auc = 0
  with torch.no_grad():    
        test_pred = []
        test_true = [] 
        for jdx, (index, data, targets) in enumerate(train_loader_at_eval):
            test_data, test_label = data.to(device), targets.to(device)
            test_data=test_data.repeat(1,3,1,1)
#             test_labels=test_labels.float()
#             test_data = test_data.to(device)
            y_pred = model(test_data)
            test_pred.append(y_pred.cpu().detach().numpy())
            test_true.append(test_label.cpu().detach().numpy())
        
        test_true = np.concatenate(test_true)
        test_pred = np.concatenate(test_pred)
        #test_pred = [1 if x > 0.5 else 0 for x in test_pred]
        val_auc =  roc_auc_score(test_true, test_pred) 
        #model.train(
        if best_val_auc < val_auc:
           best_val_auc = val_auc
        #if epoch==99:
           torch.save(model, 'aucm_trained_model_pneumoniamnist.pth')
  scheduler.step(val_auc)
  lr_scheduler.step()
  #scheduler.step()
  print ('Epoch=%s, BatchID=%s, Val_AUC=%.7f, lr=%.15f'%(epoch, idx, val_auc,  optimizer.lr))
  #for param_group in optimizer.param_groups:
   #     print('Epoch {}, LR: {}'.format(epoch, param_group['lr']))
print ('Best Val_AUC is %.4f'%best_val_auc)

        

Validating epoch: 1 Loss: -0.005301535129547119
Epoch=0, BatchID=217, Val_AUC=0.9927180, lr=0.010000000000000
Validating epoch: 2 Loss: 0.05397621542215347
Epoch=1, BatchID=217, Val_AUC=0.9933968, lr=0.010000000000000
Validating epoch: 3 Loss: 0.06452037394046783
Epoch=2, BatchID=217, Val_AUC=0.9906172, lr=0.010000000000000
Validating epoch: 4 Loss: 0.04903373494744301
Epoch 00004: reducing learning rate of group 0 to 5.0000e-03.
Epoch=3, BatchID=217, Val_AUC=0.9932174, lr=0.010000000000000
Validating epoch: 5 Loss: 0.002605720888823271
Epoch=4, BatchID=217, Val_AUC=0.9933496, lr=0.005000000000000
Validating epoch: 6 Loss: 0.02404678612947464
Epoch=5, BatchID=217, Val_AUC=0.9936981, lr=0.005000000000000
Validating epoch: 7 Loss: 0.012082899920642376
Epoch=6, BatchID=217, Val_AUC=0.9938798, lr=0.005000000000000
Validating epoch: 8 Loss: 0.012968412600457668
Epoch=7, BatchID=217, Val_AUC=0.9936849, lr=0.005000000000000
Validating epoch: 9 Loss: 0.02153426595032215
Epoch=8, BatchID=217, V

In [30]:
print(f'Testing ...')  
PATH = 'aucm_trained_model_pneumoniamnist.pth' 
model = torch.load(PATH)
#model = MyModel()  # Create an instance of your model
##model=torch.load(model_dumped)  # Load the saved parameters into the model
best_val_auc=0.0
with torch.no_grad():    
     test_pred = []
     test_true = [] 
     for jdx, (index, data, targets) in enumerate(test_loader):
         test_data, test_labels = data.to(device), targets.to(device)
         test_data=test_data.repeat(1,3,1,1)
#          test_labels=test_labels.float()
#          test_data = test_data.to(device)
         y_pred = model(test_data)
         test_pred.append(y_pred.cpu().detach().numpy())
         test_true.append(test_labels.cpu().detach().numpy())
 
     test_true = np.concatenate(test_true)
     test_pred = np.concatenate(test_pred)
     #test_pred = [1 if x > 0.5 else 0 for x in test_pred]
     #print(test_pred)
     #print(test_true)
     val_auc_mean =  roc_auc_score((test_true), test_pred) 
    # model.train
    # if best_val_auc < val_auc_mean:
    #    best_val_auc = val_auc_mean
    #    #torch.save(model.state_dict(), 'ce_pretrained_model.pth')
     print('Test_AUC=%.7f'%( val_auc_mean))

Testing ...
Test_AUC=0.9733947


Reproducibility check

In [31]:
 PATH = '/kaggle/input/pneumoniamnist-m/aucm_trained_model_pneumoniamnist.pth'
print(f'Testing ...')  
#PATH = 'aucm_trained_model_breastmnist_gaussian.pth' 
 model = torch.load(PATH)
#model = MyModel()  # Create an instance of your model
##model=torch.load(model_dumped)  # Load the saved parameters into the model
best_val_auc=0.0
with torch.no_grad():    
     test_pred = []
     test_true = [] 
     for jdx, (index, data, targets) in enumerate(test_loader):
         test_data, test_labels = data.to(device), targets.to(device)
         test_data=test_data.repeat(1,3,1,1)
#          test_labels=test_labels.float()
#          test_data = test_data.to(device)
         y_pred = model(test_data)
         test_pred.append(y_pred.cpu().detach().numpy())
         test_true.append(test_labels.cpu().detach().numpy())
 
     test_true = np.concatenate(test_true)
     test_pred = np.concatenate(test_pred)
     #test_pred = [1 if x > 0.5 else 0 for x in test_pred]
     #print(test_pred)
     #print(test_true)
     val_auc_mean =  roc_auc_score(test_true, test_pred) 
    # model.train
    # if best_val_auc < val_auc_mean:
    #    best_val_auc = val_auc_mean
    #    #torch.save(model.state_dict(), 'ce_pretrained_model.pth')
     print('Test_AUC=%.7f'%( val_auc_mean))

Testing ...
Test_AUC=0.9733947
